In [32]:
import pandas as pd
from googleapiclient.discovery import build

DEVELOPER_KEY = ''
YOUTUBE_API_SERVICE_NAME='youtube'
YOUTUBE_API_VERSION='v3'

youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=DEVELOPER_KEY)


# API를 사용하여 채널의 모든 동영상의 ID와 좋아요 갯수를 가져옵니다.
def get_channel_videos(channel_id):
    stats_dict = {}
    stats_list = []
    next_page_token = None
    
    while True:
        res = youtube.search().list(
            part='id',
            channelId=channel_id,
            maxResults=50,
            pageToken=next_page_token,
            type='video'
        ).execute()
        
        video_ids = []
        for item in res['items']:
            video_ids.append(item['id']['videoId'])
        
        # API를 사용하여 동영상의 정보를 가져옵니다.
        video_details_res = youtube.videos().list(
            part='id,snippet,statistics',
            id=','.join(video_ids),
            maxResults=50,
        ).execute()
        
        for video in video_details_res['items']:
            url_pk=video['id']
            title=video['snippet']['title']
            description=video['snippet']['description']
            view_count=video['statistics'].get('viewCount',0)
            like_count=video['statistics'].get('likeCount',0)
            published=video['snippet']['publishedAt']
            stats_dict=dict(url_pk=url_pk, title=title, description=description, published=published, view_count=view_count, like_count=like_count)
            stats_list.append(stats_dict)
        
        next_page_token = res.get('nextPageToken')
        
        if next_page_token is None:
            break
    
    return stats_list

def create_csv_and_model(video_data):
    df=pd.DataFrame(video_data)
    df.to_csv("/Users/cslee/vscode/cancook-backend/csv/자취요리신.csv")

# 채널 ID를 지정합니다.
channel_id = 'UCC9pQY_uaBSa0WOpMNJHbEQ'

# 채널의 모든 영상의 ID와 좋아요 갯수를 가져옵니다.
video_data = get_channel_videos(channel_id)
create_csv_and_model(video_data)